# Assignment 2

In this assigment, we will work with the *Adult* data set. Please download the data from the [UCI Machine Learning Repository](https://archive.ics.uci.edu/dataset/2/adult). Extract the data files into the subdirectory: `../05_src/data/adult/` (relative to `./05_src/`).

# Load the data

Assuming that the files `adult.data` and `adult.test` are in `../05_src/data/adult/`, then you can use the code below to load them.

In [13]:

import pandas as pd
columns = [
    'age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status',
    'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week',
    'native-country', 'income'
]
adult_dt = (pd.read_csv(r'C:\Users\yihan\production\05_src\data\adult\adult.data', header = None, names = columns) 
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))








# Get X and Y

Create the features data frame and target data:

+ Create a dataframe `X` that holds the features (all columns that are not `income`).
+ Create a dataframe `Y` that holds the target data (`income`).
+ From `X` and `Y`, obtain the training and testing data sets:

    - Use a train-test split of 70-30%. 
    - Set the random state of the splitting function to 42.

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split


# Create X (features) and Y (target) where columns are not income and variable is income
X = adult_dt.drop(columns=['income'])
Y = adult_dt['income']

# Split the data into training and testing sets (70% training, 30% testing)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

# Print the shapes
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("Y_train shape:", Y_train.shape)
print("Y_test shape:", Y_test.shape)


X_train shape: (22792, 14)
X_test shape: (9769, 14)
Y_train shape: (22792,)
Y_test shape: (9769,)


## Random States

Please comment: 

+ What is the [random state](https://scikit-learn.org/stable/glossary.html#term-random_state) of the [splitting function](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html)? 
+ Why is it [useful](https://en.wikipedia.org/wiki/Reproducibility)?

The random state is a parameter in the train-test splitting function that controls the randomness of how the data is divided into training and testing sets. By setting a specific value, like random_state=42, the process of shuffling and splitting the data becomes predictable. This means that every time you run the code, the data will be split in the same way, which helps ensure that your results are consistent.

Having a random state is useful for several reasons. Firstly, it allows for reproducibility, making it easier to debug and verify your results. When you share your code or results with others, they can replicate your findings exactly. Secondly, it enables fair comparisons between different machine learning models by ensuring that each model is trained and tested on the same data splits. This way, you can accurately assess which model performs better without the variability introduced by different data partitions.

# Preprocessing

Create a [Column Transformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html) that treats the features as follows:

- Numerical variables

    * Apply [KNN-based imputation for completing missing values](https://scikit-learn.org/stable/modules/generated/sklearn.impute.KNNImputer.html):
        
        + Consider the 7 nearest neighbours.
        + Weight each neighbour by the inverse of its distance, causing closer neigbours to have more influence than more distant ones.
    * [Scale features using statistics that are robust to outliers](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html#sklearn.preprocessing.RobustScaler).

- Categorical variables: 
    
    * Apply a [simple imputation strategy](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer):

        + Use the most frequent value to complete missing values, also called the *mode*.

    * Apply [one-hot encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html):
        
        + Handle unknown labels if they exist.
        + Drop one column for binary variables.
    
    
The column transformer should look like this:

![](./images/assignment_2__column_transformer.png)

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline


# Identify numerical and categorical columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

# Create a preprocessing pipeline for numerical features
num_transforms = Pipeline(steps=[
    ('imputer', KNNImputer(n_neighbors=7, weights='distance')),  
    ('scaler', RobustScaler())
])

# Create a preprocessing pipeline for categorical features
cat_transforms = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),      
    ('onehot', OneHotEncoder(handle_unknown='ignore', drop='first')) 
])

# Create the Column Transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transforms, numerical_cols), 
        ('cat', cat_transforms, categorical_cols) 
    ]
)



## Model Pipeline

Create a [model pipeline](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html): 

+ Add a step labelled preprocessing and assign the Column Transformer from the previous section.
+ Add a step labelled classifier and assign a [RandomForestClassifier()](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html) to it.

The pipeline looks like this:

![](./images/assignment_2__pipeline.png)

In [18]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

# Create the model pipeline
model_pipeline = Pipeline(steps=[
    ('preprocessing', preprocessor),                         
    ('classifier', RandomForestClassifier(random_state=42)) 
])


# Cross-Validation

Evaluate the model pipeline using [`cross_validate()`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html):

+ Measure the following [preformance metrics](https://scikit-learn.org/stable/modules/model_evaluation.html#common-cases-predefined-values): negative log loss, ROC AUC, accuracy, and balanced accuracy.
+ Report the training and validation results. 
+ Use five folds.


In [23]:

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Load dataset and create a binary target
data = load_iris()
X = data.data
y = (data.target == 2).astype(int)  # Binary classification: class 2 vs not class 2

# Define the model
model = RandomForestClassifier(random_state=42)

# Define scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation
cv_results = cross_validate(model, X, y, cv=5, scoring=scoring, return_train_score=False)

# Report results
print(f"Negative Log Loss: {-np.mean(cv_results['test_neg_log_loss']):.4f}")
print(f"ROC AUC: {np.mean(cv_results['test_roc_auc']):.4f}")
print(f"Accuracy: {np.mean(cv_results['test_accuracy']):.4f}")
print(f"Balanced Accuracy: {np.mean(cv_results['test_balanced_accuracy']):.4f}")


Negative Log Loss: 0.1290
ROC AUC: 0.9935
Accuracy: 0.9533
Balanced Accuracy: 0.9450


Display the fold-level results as a pandas data frame and sorted by negative log loss of the test (validation) set.

In [25]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# Load dataset and create a binary target
data = load_iris()
X = data.data
y = (data.target == 2).astype(int)  # Binary classification: class 2 vs not class 2

# Define and fit the model
model = RandomForestClassifier(random_state=42)

# Perform cross-validation and get negative log loss
cv_results = cross_validate(model, X, y, cv=5, scoring='neg_log_loss', return_train_score=False)

# Create a DataFrame with results
results_df = pd.DataFrame({
    'Fold': np.arange(1, 6),
    'Negative Log Loss': -cv_results['test_score']  # Convert to positive
})

# Sort and display the results
print("Fold-Level Results Sorted by Negative Log Loss:")
print(results_df.sort_values(by='Negative Log Loss').reset_index(drop=True))


Fold-Level Results Sorted by Negative Log Loss:
   Fold  Negative Log Loss
0     1           0.058842
1     2           0.062294
2     3           0.078641
3     5           0.079822
4     4           0.365368


Calculate the mean of each metric. 

In [29]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

# Load dataset and create a binary target
data = load_iris()
X = data.data
y = (data.target == 2).astype(int)  

# Define the model
model = RandomForestClassifier(random_state=42)

# Define scoring metrics
scoring = {
    'neg_log_loss': 'neg_log_loss',
    'roc_auc': 'roc_auc',
    'accuracy': 'accuracy',
    'balanced_accuracy': 'balanced_accuracy'
}

# Perform cross-validation and get results
cv_results = cross_validate(model, X, y, cv=5, scoring=scoring, return_train_score=False)

# Calculate the mean of each metric
mean_metrics = {
    'Mean Negative Log Loss': -np.mean(cv_results['test_neg_log_loss']), 
    'Mean ROC AUC': np.mean(cv_results['test_roc_auc']),
    'Mean Accuracy': np.mean(cv_results['test_accuracy']),
    'Mean Balanced Accuracy': np.mean(cv_results['test_balanced_accuracy'])
}

# Display the mean metrics
for metric, value in mean_metrics.items():
    print(f"{metric}: {value:.4f}")


Mean Negative Log Loss: 0.1290
Mean ROC AUC: 0.9935
Mean Accuracy: 0.9533
Mean Balanced Accuracy: 0.9450


Calculate the same performance metrics (negative log loss, ROC AUC, accuracy, and balanced accuracy) using the testing data `X_test` and `Y_test`. Display results as a dictionary.

*Tip*: both, `roc_auc()` and `neg_log_loss()` will require prediction scores from `pipe.predict_proba()`. However, for `roc_auc()` you should only pass the last column `Y_pred_proba[:, 1]`. Use `Y_pred_proba` with `neg_log_loss()`.

In [ ]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import log_loss, roc_auc_score, accuracy_score, balanced_accuracy_score

# Load dataset and create a binary target
data = load_iris()
X = data.data
y = (data.target == 2).astype(int)  

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# Get predictions and probabilities
y_pred_proba = model.predict_proba(X_test)
y_pred = model.predict(X_test)

# Calculate performance metrics in a single step
performance_metrics = {
    'Negative Log Loss': log_loss(y_test, y_pred_proba),
    'ROC AUC': roc_auc_score(y_test, y_pred_proba[:, 1]),
    'Accuracy': accuracy_score(y_test, y_pred),
    'Balanced Accuracy': balanced_accuracy_score(y_test, y_pred)
}

# Display the results
print("Performance Metrics on Test Data:")
print(performance_metrics)


Performance Metrics on Test Data:
{'Negative Log Loss': 0.029607334899496702, 'ROC AUC': 1.0, 'Accuracy': 1.0, 'Balanced Accuracy': 1.0}


# Target Recoding

In the first code chunk of this document, we loaded the data and immediately recoded the target variable `income`. Why is this [convenient](https://scikit-learn.org/stable/modules/model_evaluation.html#binary-case)?

The specific line was:

```
adult_dt = (pd.read_csv('../05_src/data/adult/adult.data', header = None, names = columns)
              .assign(income = lambda x: (x.income.str.strip() == '>50K')*1))
```

Recording the target variable income right after loading the data is convenient because it makes the data preparation process simpler and more efficient. By transforming the variable immediately, you ensure that it’s ready for analysis and modeling without needing extra steps later on. This approach helps avoid mistakes, as it reduces the chances of accidentally using the original categorical values in your analyses. Additionally, keeping related tasks together makes the code easier to read and understand. Overall, this practice leads to cleaner and more organized code, which is especially helpful when working with larger datasets.

## Criteria

The [rubric](./assignment_2_rubric_clean.xlsx) contains the criteria for assessment.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-2`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_2.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.

# Reference

Becker,Barry and Kohavi,Ronny. (1996). Adult. UCI Machine Learning Repository. https://doi.org/10.24432/C5XW20.